In [1]:
import numpy as np
import matplotlib.pyplot as plt
import optics, utils
import tifffile
from tqdm import tqdm

def plot_fourier(fourier):
    plt.imshow(np.log(1+np.abs(fourier)))

In [4]:
import napari
viewer = napari.Viewer()

<h1> 1. Field retrieval from hologram </h1>

In [43]:
background = tifffile.imread('yeast/background.tif')
sample = tifffile.imread('yeast/sample.tif')

N = background[0].shape[0]
Z = background.shape[0]

In [44]:
viewer.add_image(sample)

<Image layer 'sample' at 0x1fb00d139e0>

In [45]:
m = 1
cm = 1e-2
mm = 1e-3
um = 1e-6
nm = 1e-9

# Laser configuration
lam = 532 * nm
NA = 1.2
# n_medium = 1
# dx_ol = lam / 4 / NA

n_medium = 1.33
dx_ol = 0.083 * um

cutoff = 1/3

<h3> object_center gives frequency coordinate of illumination beam in fourier space </h3>

\begin{gather*}


I(\vec{r}) = |R+U|^2 \\
\hat{I(\vec{\nu})} = \mathcal{F} [I(\vec{r})]\\
\mathcal{P} [\hat{I(\vec{\nu})}] = center[cut[\hat{I(\vec{\nu})}]]= center [\mathcal{F} [R^* U]] = \mathcal{F} [U] \\
\Downarrow \\
U(\vec{r}) = \mathcal{F}^{-1} [\mathcal{F} [U]] 


\end{gather*}

<h3> Get object field U : F[(DC term) + UR* + U*R] -> F[U] -> U</h3>

In [66]:
temp_background_object_field, temp_sample_object_field = optics.Holography_Off_Axis.get_object_field(background_hologram=background, 
                                                                                           sample_hologram=sample)

background_object_field = temp_background_object_field / temp_background_object_field
sample_object_field = temp_sample_object_field / temp_background_object_field

100%|██████████| 49/49 [00:01<00:00, 25.19it/s]


<h1> Calculate spatial frequency of the illumination beam and create frequency coordinates space </h1>

In [67]:
# Get optical parameters
params = utils.get_optical_parameters(image_shape=(N, N), lam=lam, n_medium=n_medium, dx_cam=3.5*um, dx_ol=dx_ol)
illumination_frequency = dict()
fourier_coordinates = dict()
params

{'dx_cam': 3.5e-06,
 'B_cam': 142857.14285714287,
 'dv_cam': 525.2100840336135,
 'dx_ol': 8.3e-08,
 'B_ol': 6024096.385542168,
 'dv_ol': 22147.413182140324,
 'v0': 1879699.2481203005,
 'v_nm': 2499999.9999999995,
 'k_nm': 15707963.267948963,
 's_nm': 112.0}

In [68]:
# Calculate spatial frequency coordinates space
v0x, v0y, v0z = [], [], []

for i in range(Z):
    sample_freq_idx = np.array(
        utils.get_maxindex(
            np.abs(np.fft.fftshift(np.fft.fft2(np.fft.ifftshift(temp_sample_object_field[i]))))
            )
        )
    if i==0:
        center_idx = sample_freq_idx
        
    incident_freq = (sample_freq_idx - center_idx) * params['dv_ol']
    v0x.append(incident_freq[1])
    v0y.append(incident_freq[0])
    v0z.append(round(np.sqrt(params['v_nm']**2 - incident_freq[1]**2 - incident_freq[0]**2)))

illumination_frequency['v0x'] = v0x.copy()
illumination_frequency['v0y'] = v0y.copy()
illumination_frequency['v0z'] = v0z.copy()

In [69]:
# Calculate fourier coordinates
v_x = np.arange(-N//2, N//2) * params['dv_ol']
v_y = np.arange(-N//2, N//2) * params['dv_ol']
V_x, V_y = np.meshgrid(v_x, v_y)

V_z = params['v_nm']**2 - V_x**2 - V_y**2
V_z[V_z<0] = 0
V_z = np.sqrt(V_z)

fourier_coordinates['V_x'] = V_x.copy()
fourier_coordinates['V_y'] = V_y.copy()
fourier_coordinates['V_z'] = V_z.copy()

<h1> The first Born </h3>

In [70]:
born_scattered_field = sample_object_field - 1

In [71]:
# Set recon space height
H = 300

# Initialize recon frequency space v_z coordinates
fourier_coordinates['recon_V_z'] = np.arange(-H//2, H//2) * params['dv_ol']
recon_V_z_origin_z_idx = np.where(fourier_coordinates['recon_V_z']==0)[0][0]

# Initialize recon field and auxillary arrays
born = np.zeros((N, N, H), dtype=np.complex128)
count = np.zeros((N, N, H))
NA_circle = utils.circular_filter((N, N), pixel_radius=int(params['v_nm']//params['dv_ol']))

# Reconstruction
for i in tqdm(range(Z)):
    # Shifted coordinates
    shifted_V_x = fourier_coordinates['V_x'] - illumination_frequency['v0x'][i]
    shifted_V_y = fourier_coordinates['V_y'] - illumination_frequency['v0y'][i]
    shifted_V_z = np.roll(fourier_coordinates['V_z'], shift=-round(illumination_frequency['v0x'][i]//params['dv_ol']), axis=1)
    shifted_V_z = np.roll(shifted_V_z, shift=-round(illumination_frequency['v0y'][i]//params['dv_ol']), axis=0)
            
    # Scattered field
    Us = born_scattered_field[i].copy()
    fourier_Us = np.fft.fftshift(np.fft.fft2(np.fft.ifftshift(Us))) * params['dx_ol'] * params['dx_ol']
    fourier_Us = fourier_Us * NA_circle
    # Since we've already divided sample field by background field before, there's no need to roll the fourier field.
    # fourier_Us = np.roll(fourier_Us, shift=-s0y[i]+1, axis=0)
    # fourier_Us = np.roll(fourier_Us, shift=-s0x[i], axis=1)
            
    # Coefficients
    coeff = shifted_V_z / 1j
            
    # Field
    F_born = coeff * fourier_Us
        
    # Ewald sphere projection preparation    
    yx_idx = np.where((shifted_V_z>0) & (NA_circle != 0) & 
                    (shifted_V_x > -(N//2) * params['dv_ol']) & (shifted_V_x < (N//2) * params['dv_ol']) &
                    (shifted_V_y > -(N//2) * params['dv_ol']) & (shifted_V_y < (N//2) * params['dv_ol'])
                )
    ## Move to the origin 
    z_idx = np.round((shifted_V_z[yx_idx]-shifted_V_z[N//2, N//2])//params['dv_ol'] + recon_V_z_origin_z_idx).astype(int)
    yxz_idx = (yx_idx[0], yx_idx[1], z_idx)
    
    # Ewald sphere projection with count array (to avoid multiple addition of the potential)
    count[yxz_idx] += np.ones((N, N, H))[yxz_idx]
    born[yxz_idx] += F_born[yx_idx]  
    
born[count==0] = 0
born[count!=0] = born[count!=0]/count[count!=0]
born = born / params['dx_ol'] / params['dx_ol'] / params['dx_ol']

100%|██████████| 49/49 [00:07<00:00,  6.99it/s]


In [72]:
potential = np.fft.ifftshift(np.fft.ifftn(np.fft.fftshift(born)))
ri = np.sqrt(1 + 4 * np.pi * potential / params['k_nm']**2) * n_medium
viewer.add_image(np.real(ri))

<Image layer 'Image [1]' at 0x1fb140eb590>

<h1> The first Rytov </h1>

In [76]:
rytov_scattered_field = np.log(np.abs(sample_object_field)) + 1j * np.angle(sample_object_field)

In [98]:
# Set recon space height
H = 300

# Initialize recon frequency space v_z coordinates
fourier_coordinates['recon_V_z'] = np.arange(-H//2, H//2) * params['dv_ol']
recon_V_z_origin_z_idx = np.where(fourier_coordinates['recon_V_z']==0)[0][0]

# Initialize recon field and auxillary arrays
rytov = np.zeros((N, N, H), dtype=np.complex128)
count = np.zeros((N, N, H))
NA_circle = utils.circular_filter((N, N), pixel_radius=int(params['v_nm']//params['dv_ol']))

# Reconstruction
for i in tqdm(range(Z)):
    # Shifted coordinates
    shifted_V_x = fourier_coordinates['V_x'] - illumination_frequency['v0x'][i]
    shifted_V_y = fourier_coordinates['V_y'] - illumination_frequency['v0y'][i]
    shifted_V_z = np.roll(fourier_coordinates['V_z'], shift=-round(illumination_frequency['v0x'][i]//params['dv_ol']), axis=1)
    shifted_V_z = np.roll(shifted_V_z, shift=-round(illumination_frequency['v0y'][i]//params['dv_ol']), axis=0)

    # Scattered field
    Us = rytov_scattered_field[i].copy()
    fourier_Us = np.fft.fftshift(np.fft.fft2(np.fft.ifftshift(Us))) * params['dx_ol'] * params['dx_ol']
    fourier_Us = fourier_Us * NA_circle
    
    # Since we've already divided sample field by background field before, there's no need to roll the fourier field.
    # fourier_Us = np.roll(fourier_Us, shift=-s0y[i]+1, axis=0)
    # fourier_Us = np.roll(fourier_Us, shift=-s0x[i], axis=1)
            
    # Coefficients
    coeff = shifted_V_z / 1j
            
    # Field
    F_rytov = coeff * fourier_Us
    
    # Ewald sphere projection preparation    
    yx_idx = np.where((shifted_V_z>0) & (NA_circle != 0) & 
                    (shifted_V_x > -(N//2) * params['dv_ol']) & (shifted_V_x < (N//2) * params['dv_ol']) &
                    (shifted_V_y > -(N//2) * params['dv_ol']) & (shifted_V_y < (N//2) * params['dv_ol'])
                )
    ## Move to the origin 
    z_idx = np.round((shifted_V_z[yx_idx]-shifted_V_z[N//2, N//2])//params['dv_ol'] + recon_V_z_origin_z_idx).astype(int)
    yxz_idx = (yx_idx[0], yx_idx[1], z_idx)
    
    # Ewald sphere projection with count array (to avoid multiple addition of the potential)
    count[yxz_idx] += np.ones((N, N, H))[yxz_idx]        
    rytov[yxz_idx] += F_rytov[yx_idx]
    
rytov[count==0] = 0
rytov[count!=0] = rytov[count!=0] / count[count!=0]
rytov = rytov / params['dx_ol'] / params['dx_ol'] / params['dx_ol']

100%|██████████| 49/49 [00:06<00:00,  7.02it/s]


In [99]:
potential = np.fft.ifftshift(np.fft.ifftn(np.fft.fftshift(rytov)))
ri = np.sqrt(1 + 4 * np.pi * potential / params['k_nm']**2) * n_medium
viewer.add_image(np.real(ri))

<Image layer 'Image' at 0x1fb1407b3b0>

In [100]:
viewer.add_image(np.real(ri))

<Image layer 'Image' at 0x1fb182db320>